# Getting Started

## Wrapping a PyTorch model
Create a simple PyTorch model.

In [8]:
import torch
import numpy as np
import torch.nn as nn

class MLP(nn.Module):
    """
    Simple MLP.
    """
    def __init__(self, input_dim, output_dim, hidden_dim):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.mlp(x)

class SimpleModel(nn.Module):
    """
    Simple model class.
    """
    def __init__(self, input_dim, output_dim, hidden_dim = 64):
        super(SimpleModel, self).__init__()

        self.mlp = MLP(input_dim, output_dim, hidden_dim)

    def forward(self, x):
        x = self.mlp(x)
        return x

Train the model on some data.

$$
y = x_0^2 +3 \sin(x_4)-2
$$

In [9]:
# Make the dataset 
x = np.array([np.random.uniform(0, 1, 10_000) for _ in range(5)]).T  
y = x[:, 0]**2 + 3*np.sin(x[:, 4]) - 4
noise = np.array([np.random.normal(0, 0.05*np.std(y)) for _ in range(len(y))])
y = y + noise 

In [10]:
# Set up training

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

def train_model(model, dataloader, opt, criterion, epochs = 100):
    """
    Train a model for the specified number of epochs.
    
    Args:
        model: PyTorch model to train
        dataloader: DataLoader for training data
        opt: Optimizer
        criterion: Loss function
        epochs: Number of training epochs
        
    Returns:
        tuple: (trained_model, loss_tracker)
    """
    loss_tracker = []
    for epoch in range(epochs):
        epoch_loss = 0.0
        
        for batch_x, batch_y in dataloader:
            # Forward pass
            pred = model(batch_x)
            loss = criterion(pred, batch_y)
            
            # Backward pass
            opt.zero_grad()
            loss.backward()
            opt.step()
            
            epoch_loss += loss.item()
        
        loss_tracker.append(epoch_loss)
        if (epoch + 1) % 5 == 0:
            avg_loss = epoch_loss / len(dataloader)
            print(f'Epoch [{epoch+1}/{epochs}], Avg Loss: {avg_loss:.6f}')
    return model, loss_tracker

# Instantiate the model
model = SimpleModel(input_dim=x.shape[1], output_dim=1)

# Set up training
criterion = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=0.001)
X_train, _, y_train, _ = train_test_split(x, y.reshape(-1,1), test_size=0.2, random_state=290402)

# Set up dataset
dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [11]:
# Train the model and save the weights

model, losses = train_model(model, dataloader, opt, criterion, 20)
torch.save(model.state_dict(), 'model_weights.pth')

Epoch [5/20], Avg Loss: 0.087841
Epoch [10/20], Avg Loss: 0.060187
Epoch [15/20], Avg Loss: 0.048619
Epoch [20/20], Avg Loss: 0.039380


Wrap the mlp layer in the trained model with MLP_SR.

In [13]:
from symtorch.mlp_sr import MLP_SR
model.mlp = MLP_SR(model.mlp, mlp_name = 'Sequential')

## Interpret the MLP

In this example, we pass extra parameters into the `.interpret` method (complexity of operators/constants and parsimony, which is a penalisation of complexity).\
To see all the possible parameters, please see the `PySRRegressor` class from [PySR](https://ai.damtp.cam.ac.uk/pysr/api/).

In this example, we turn verbosity off because we are in a Jupyter notebook. For best performance, run in IPython, as you can terminate the SR any time.

In [15]:
model.mlp.interpret(torch.FloatTensor(X_train), 
                       niterations = 500, # Should set to higher
                       verbosity=0, 
                       complexity_of_operators = {"sin":3, "exp":3}, 
                       complexity_of_constants = 2,
                       parsimony = 0.1)

/Users/liz/PhD/InterpretSR_project/interpretsr_venv/lib/python3.11/site-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


❤️ SR on Sequential complete.
💡Best equation found to be ((x0 * x0) * sin(x4 + 1.324108)) + ((sin(x4) * 3.0265517) + -3.9891667).


,model_selection,'best'
,binary_operators,"['+', '*']"
,unary_operators,"['inv(x) = 1/x', 'sin', ...]"
,expression_spec,None
,niterations,500
,populations,31
,population_size,27
,max_evals,None
,maxsize,30
,maxdepth,None
,warmup_maxsize_by,None


See the full Pareto front of equations. The best equation is chosen as a balance of accuracy and complexity.\
Outputs from *PySR* are saved in `SR_output/MLP_name`.

In [17]:
print(model.mlp.pysr_regressor)

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                                 x4   
	1         2.573184                                         -2.2818904   
	2         0.397069                                    x4 + -2.7839313   
	3         0.472875                              x4 + (x4 + -3.285883)   
	4         0.232451                      (x4 + -1.4082903) * 2.5177994   
	5         0.976004                      x0 + ((x4 + -3.7878833) + x4)   
	6         1.055289                (x4 * 2.5016766) + (x0 + -4.039698)   
	7         0.141551        ((x4 * 2.5023715) + -3.8732893) + (x0 * x0)   
	8         0.365466           (x0 + -4.136783) + (sin(x4) * 2.9319415)   
	9         0.303630   ((x0 * x0) + (sin(x4) * 2.9325113)) + -3.9702833   
	10        0.024581  ((x0 * x0) + -4.038742) + (x4 * ((x4 * -0.9855...   
	11        0.403158  (((x0 * (x0 * inv(x4))) + 2.9950936) * sin(x4)...   
	12      

## Switch to Using the Equation Instead in the Forwards Pass

You can choose the equation you want to switch to by choosing the desired complexity of equation. \
If left blank, then we choose the best equation chosen by *PySR*.

In [18]:
model.mlp.switch_to_equation(complexity=14) 

✅ Successfully switched Sequential to symbolic equation: ((x0 * x0) + (sin(x4) * 2.9325113)) + -3.9702833
📊 Using variables: ['x0', 'x4'].


Now when running the forwards pass through the model, it uses the symbolic equation instead of the MLP. 

In [20]:
interpretable_outputs = model.mlp(torch.tensor(X_train, dtype=torch.float32))
interpretable_outputs

tensor([[-3.8636],
        [-3.4790],
        [-1.5592],
        ...,
        [-2.7508],
        [-2.6754],
        [-3.3049]])

## Switch to Using the MLP in the Forwards Pass

In [21]:
mlp_outputs = model.mlp.switch_to_mlp()

✅ Switched Sequential back to MLP


In [22]:
model_outputs = model.mlp(torch.tensor(X_train, dtype=torch.float32))
model_outputs

tensor([[-3.8313],
        [-3.5179],
        [-1.5196],
        ...,
        [-2.7176],
        [-2.6945],
        [-3.3658]], grad_fn=<AddmmBackward0>)

In [23]:
# Clean up 
import os
import shutil
if os.path.exists('SR_output'):
    shutil.rmtree('SR_output')
os.remove('model_weights.pth')